## Stock Market Analysis

In this notebook, we will discover and explore data from the stock market, particularly some dead stocks. We will see how to use UpstoxApi by [Upstox](https://upstox.com/) to get stock historical data, for 1 min chart(open, high, low, close) and export it into CSV format

### Getting the Instrument details

* instrument_key	-->	The unique identifier used across Upstox APIs for instrument identification.
* exchange_token	-->	The numerical identifier issued by the exchange representing the instrument.
* tradingsymbol	    -->	Shows the trading symbol which could be a combination of symbol name, instrument, expiry date etc.
* name	            -->	Name of the company (for equity instruments).
* last_price	    -->	Last traded price.
* expiry	        -->	Expiry date (for derivatives). Data format is yyyy-MM-dd
* strike	        -->	Indicates the predetermined price at which an option can be bought or sold when it's exercised.
* tick_size	        -->	Measure of the minimum upward or downward movement in the price of an instrument.
* lot_size	        -->	Minimum size in which the stock futures or index futures can be traded.
* instrument_type	-->	Instrument type of a particular contract. Possible values: FUTSTK, OPTSTK, OPTIDX etc.
* option_type	    -->	Option type of the option contracts (applicable only for options contract). Possible values: CE, PE
* exchange	        -->	Exchange to which the order is associated.Possible values: NSE_EQ, NSE_FO, NCD_FO, NSE_INDEX, BSE_EQ, BSE_FO, BCD_FO, BSE_INDEX, MCX_FO, NSE_COM etc.

In [13]:
import gzip
import json

# Path to the compressed JSON file
file_path = "NSE.json.gz"

# Function to extract and read JSON data
def extract_json_data(file_path):
    with gzip.open(file_path, 'rt', encoding='utf-8') as file:
        data = json.load(file)
    return data

# Extract data
data = extract_json_data(file_path)

# Function to filter by segment
def filter_by_segment(data, segment=None, name=None, instrument_type=None, 
                      instrument_key=None, exchange_token=None, trading_symbol=None):
    
    filtered_data = []
    for item in data:
        if (
            (segment is None or item.get("segment") == segment)
            and (name is None or item.get("name") == name)
            and (instrument_type is None or item.get("instrument_type") == instrument_type)
            and (instrument_key is None or item.get("instrument_key") == instrument_key)
            and (exchange_token is None or item.get("exchange_token") == exchange_token)
            and (trading_symbol is None or item.get("trading_symbol") == trading_symbol)
        ):
            filtered_data.append(item)

    return filtered_data

# Example: Filter for 'NSE_EQ' segment
filteredTicker = filter_by_segment(data, "NSE_EQ", "AHLADA ENGINEERS LIMITED" , "EQ", None, None, None)

# Print the records from the filtered data
print(json.dumps(filteredTicker, indent=3))

instrument_key = filteredTicker[0]["instrument_key"]
print(instrument_key)

[
   {
      "segment": "NSE_EQ",
      "name": "AHLADA ENGINEERS LIMITED",
      "exchange": "NSE",
      "isin": "INE00PV01013",
      "instrument_type": "EQ",
      "instrument_key": "NSE_EQ|INE00PV01013",
      "lot_size": 1,
      "freeze_quantity": 100000.0,
      "exchange_token": "2004",
      "tick_size": 1.0,
      "trading_symbol": "AHLADA",
      "short_name": "Ahlada Engineers",
      "qty_multiplier": 1.0,
      "security_type": "NORMAL"
   }
]
NSE_EQ|INE00PV01013


### Importing Flask to set up our server

In [4]:
import requests
import threading
from flask import Flask, request, jsonify, redirect, session

# Flask App
app = Flask(__name__)   
app.secret_key = "2356"

ACCESS_TOKEN = None

@app.route('/')
def login():
    return redirect("https://api.upstox.com/v2/login/authorization/dialog?response_type=code&client_id=a0126484-b118-4b8a-b26c-9c85566d4d2f&redirect_uri=http://127.0.0.1:5000/callback&state=xyz")

@app.route('/callback')
def callback():
    global ACCESS_TOKEN

    code = request.args.get("code")
    if not code:
        return "Authorization failed", 400

    payload = {
        "code": code,
        "client_id": "a0126484-b118-4b8a-b26c-9c85566d4d2f",
        "client_secret": "jm4hqianzp",
        "redirect_uri": "http://127.0.0.1:5000/callback",
        "grant_type": "authorization_code",
    }
    response = requests.post("https://api.upstox.com/v2/login/authorization/token", data=payload)

    try:
        data = response.json()
        if "access_token" not in data:
            return jsonify({"error": data}), 400

        ACCESS_TOKEN = data["access_token"]  # Store in a global variable
        session["access_token"] = ACCESS_TOKEN
        return redirect("/profile")
    
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode data response from Upstox", 500

@app.route('/profile')
def profile():
    if "access_token" not in session:
        return redirect("/")
    
    headers = {"Authorization": f"Bearer {session['access_token']}"}
    response = requests.get("https://api.upstox.com/v2/user/profile", headers=headers)

    try:
        return jsonify(response.json())
    except requests.exceptions.JSONDecodeError:
        return "Failed to decode profile response", 500

# Run Flask in a background thread (if inside Jupyter)
def run_flask():
    app.run(debug=True, use_reloader=False)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


### Making function to request data from the upstox server

In [6]:
def make_request(method, url, header=None):
    response = None
    try:
        if method == 'GET':
            request = requests.get(url, headers=header)
        else:
            raise ValueError("Not Valid HTTP Method")
        
        if request.status_code == 200:
            return request.json()   
        else:
            return response
    except request.exceptions.RequestException as e:
        print(f'Error : {e}')
        return None

### Requesting Data

In [ ]:
interval= "1minute"
from_date = "2025-02-07"
to_date = "2025-02-07"

url = f"https://api.upstox.com/v2/historical-candle/{instrument_key}/{interval}/{to_date}/{from_date}"

header = {
    'accept': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
    }

response = make_request('GET', url, header=header)  
print(response)

{'status': 'success', 'data': {'candles': [['2025-02-07T15:15:00+05:30', 86.11, 86.89, 86.11, 86.89, 292, 0], ['2025-02-07T14:45:00+05:30', 86.7, 86.89, 86, 86.11, 686, 0], ['2025-02-07T14:15:00+05:30', 86.42, 86.7, 86.42, 86.7, 546, 0], ['2025-02-07T13:45:00+05:30', 87.53, 87.53, 86.42, 86.42, 2623, 0], ['2025-02-07T13:15:00+05:30', 87.51, 87.53, 87.51, 87.53, 3, 0], ['2025-02-07T12:45:00+05:30', 87.21, 87.52, 87.21, 87.51, 53, 0], ['2025-02-07T12:15:00+05:30', 87.32, 87.32, 87.15, 87.21, 431, 0], ['2025-02-07T11:45:00+05:30', 88.13, 88.13, 87.21, 87.32, 1360, 0], ['2025-02-07T11:15:00+05:30', 88.2, 88.74, 88.13, 88.13, 733, 0], ['2025-02-07T10:45:00+05:30', 88.2, 88.69, 88.2, 88.2, 258, 0], ['2025-02-07T10:15:00+05:30', 88.21, 88.21, 88.19, 88.2, 100, 0], ['2025-02-07T09:45:00+05:30', 88.86, 88.86, 88.21, 88.21, 180, 0], ['2025-02-07T09:15:00+05:30', 89.01, 89.19, 88.21, 88.86, 868, 0]]}}


### Importing Pandas to filter/format the data

In [18]:
import pandas as pd

# Extract candle data if response is valid
candles = response.get("data", {}).get("candles", [])

if candles:
    # Define column names dynamically
    required_columns = ["timestamp", "open", "high", "low", "close", "volume"]
    
    # Create DataFrame
    df = pd.DataFrame([candle[:6] for candle in candles], columns=required_columns)
    
    # Convert timestamp to datetime
    df["timestamp"] = pd.to_datetime(df["timestamp"]).dt.strftime("%H:%M")
    
    # Display first 5 rows
    print(df.head())

  timestamp   open   high    low  close  volume
0     15:15  86.11  86.89  86.11  86.89     292
1     14:45  86.70  86.89  86.00  86.11     686
2     14:15  86.42  86.70  86.42  86.70     546
3     13:45  87.53  87.53  86.42  86.42    2623
4     13:15  87.51  87.53  87.51  87.53       3


### Extracting the data in CSV file

In [16]:
df.to_csv("AHLADA ENGINEERS LIMITED.csv", index=False)

## Conclusion

This Projects illustrates the working example of an API where the data is fetched from Upstox using it's API using Flask. The data includes the historical candle chart for 1 minute timeframe. Which is then formated and cleaned using Pandas and then extracted into Csv format.